In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
NEI = pd.read_csv(r"NEI2020_data.csv")
rwc = NEI.loc[NEI['scc'].astype(str).str.startswith(('2104008', '2104009'))]
rwc['pollutant code'] = rwc['pollutant code'].astype(str)

/home/ksz4578/.conda/envs/basic/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (3,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_162109/3815376252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwc['pollutant code'] = rwc['pollutant code'].astype(str)


In [3]:
allowed_sccs = ['72918219', 'NOX', 'VOC', 'PM25-PRI', '106990', '107028', '108952',
       '1319773', '1746016', '19408743', '3268879', '35822469',
       '39001020', '39227286', '40321764', 'NH3', 'CO', 'PM-CON',
       'PM10-PRI', 'PM10-FIL', 'SO2', 'PM25-FIL', '218019', '41637905',
       '50000', '50328', '56553', '57117314', '57117416', '57117449',
       '57653857', '60851345', '51207319', '85018', '91203', '95476',
       '108883', '120127', '129000', '191242', '192972', '193395',
       '198550', '203338', '205992', '206440', '207089', '208968',
       '53703', '67562394', '55673897', '628', '70648269', '71432',
       '7439976', '75070', '7439965', '7440020', '7440439', '83329',
       '86737', '203123', 'CO2', 'N2O', '57976', '92524', '7439921',
       '7440360', '7440382', '7440484', '7782492', '7782505', '123386',
       '130498292', '16065831', '18540299']

## check which aren't in the original RWC File
for scc in rwc['pollutant code'].unique():
    if scc not in allowed_sccs:
        rwc = rwc.loc[~(rwc['pollutant code'] == scc)]
        
## check which aren't in the original RWC File
for scc in rwc['pollutant code'].unique():
    if scc not in allowed_sccs:
        print(scc)

In [4]:
# confirms that all PM25-PRI is accounted for so can remove the other categories
print("SO4", rwc.loc[rwc['pollutant code'] == "SO4"]["total emissions"].sum())
print("NO3", rwc.loc[rwc['pollutant code'] == "NO3"]["total emissions"].sum())
print("PMFINE", rwc.loc[rwc['pollutant code'] == "PMFINE"]["total emissions"].sum())
print("pm25 pri", rwc.loc[rwc['pollutant code'] == "PM25-PRI"]["total emissions"].sum())

SO4 0.0
NO3 0.0
PMFINE 0.0
pm25 pri 485077.8252135248


In [5]:
rwc = rwc.loc[~(rwc['pollutant code'].isin(["NO3", "PMFINE", "SO4"]))]

In [6]:
rwc['country_cd'] = 'US'
rwc = rwc.rename(columns={'fips code': 'region_cd', 
                          'data set':'data_set_id', 
                          'pollutant code':'poll',
                          'total emissions':'ann_value'})
rwc['tribal_code'] = np.nan
rwc['census_tract_cd'] = np.nan
rwc['shape_id'] = np.nan
rwc['emis_type'] = np.nan
rwc['ann_pct_red'] = np.nan
rwc['control_ids'] = np.nan
rwc['control_measures'] = 'PROJECTION'
rwc['current_cost'] = np.nan
rwc['cumulative_cost'] = np.nan
rwc['projection_factor'] = 1
rwc['reg_codes'] = np.nan
rwc['calc_method'] = np.nan
rwc['calc_year'] = 2020
rwc['date_updated'] = 20240404
rwc['data_set_id'] = "2020EPA_NONPOINT"
for key in ['jan_value', 'feb_value', 'mar_value',
       'apr_value', 'may_value', 'jun_value', 'jul_value', 'aug_value',
       'sep_value', 'oct_value', 'nov_value', 'dec_value', 'jan_pctred',
       'feb_pctred', 'mar_pctred', 'apr_pctred', 'may_pctred', 'jun_pctred',
       'jul_pctred', 'aug_pctred', 'sep_pctred', 'oct_pctred', 'nov_pctred',
       'dec_pctred', 'comment']:
    rwc[key] = np.nan



In [7]:
## scale emissions (units)
rwc.loc[rwc["emissions uom"] == "LB", "ann_value"] /= 2000

In [8]:
rwc = rwc[['country_cd', 'region_cd', 'tribal_code', 'census_tract_cd', 'shape_id',
       'scc', 'emis_type', 'poll', 'ann_value', 'ann_pct_red', 'control_ids',
       'control_measures', 'current_cost', 'cumulative_cost',
       'projection_factor', 'reg_codes', 'calc_method', 'calc_year',
       'date_updated', 'data_set_id', 'jan_value', 'feb_value', 'mar_value',
       'apr_value', 'may_value', 'jun_value', 'jul_value', 'aug_value',
       'sep_value', 'oct_value', 'nov_value', 'dec_value', 'jan_pctred',
       'feb_pctred', 'mar_pctred', 'apr_pctred', 'may_pctred', 'jun_pctred',
       'jul_pctred', 'aug_pctred', 'sep_pctred', 'oct_pctred', 'nov_pctred',
       'dec_pctred', 'comment']]

In [9]:
rwc['region_cd'] = rwc['region_cd'].astype(str)
rwc['scc'] =  rwc['scc'].astype(str)

In [10]:
for pollutant in rwc['poll'].unique():
    print(pollutant, len(rwc.loc[rwc['poll'] == pollutant].index) )

SO2 43816
108883 43816
108952 43816
208968 47043
56553 22288
129000 47043
VOC 47043
106990 43816
91203 47043
85018 47043
PM25-FIL 47043
75070 43816
86737 47043
191242 22288
PM10-PRI 47043
50328 47043
PM10-FIL 47043
PM-CON 47043
1319773 43816
NH3 43930
50000 43816
83329 47043
120127 47043
NOX 47043
206440 47043
CO 47043
PM25-PRI 47043
107028 43816
203123 43816
203338 19061
7440439 12774
7440020 12774
207089 22288
53703 22288
7439965 12774
198550 19061
123386 43816
192972 19061
7439976 43932
218019 31286
71432 47043
95476 43816
205992 22288
193395 22288
41637905 19061


In [11]:
# Export headers without quotes
with open('rwc_2020.csv', 'w') as f:
    f.write(','.join(rwc.columns) + '\n')

# Export DataFrame to CSV with quotes around string values only
rwc.to_csv('rwc_2020.csv', index=False, na_rep='', quoting=csv.QUOTE_NONNUMERIC, header = True)

## Deal with tribal codes

## Deal with projection factors

## Deal with SCC codes allocation

In [ ]:
# 2104008300 --> Woodstove: freestanding, general
    # map to 2104008310 --> Woodstove: freestanding, non-EPA certified 
    # map to 2104008320 --> Woodstove: freestanding, EPA certified, non-catalytic
    # map to 2104008330 --> Woodstove: freestanding, EPA certified, catalytic
# 2104008530 --> Furnace: Indoor, pellet-fired, general 
    # map to 2104008510 --> Furnace: Indoor, cordwood-fired, non-EPA certified)
# 2104008620 --> Hydronic heater: indoor 
    # map to 2104008610 --> Hydronic heater: outdoor
# 2104008630 --> Hydronic heater: pellet-fired
    # map to 2104008610 --> Hydronic heater: outdoor

In [23]:
rwc.loc[rwc['scc'] == 2104008530,'scc'] = 2104008510
rwc.loc[(rwc['scc'] == 2104008620) | (rwc['scc'] == 2104008630),'scc'] = 2104008610

# save file

In [7]:
rwc2020 = pd.read_pickle('rwc_2020.pkl')

In [ ]:
rwc.to_csv('rwc_2020.csv')